# Image Analysis

This notebook shows some simple examples of basic image analysis techniques.

Although the code should be roughly self-explanatory, please read along with these slides (INSERT SLIDES) for a better grasp of the concepts.


# Introduction

Image analysis is an umbrella term for various methods and tools for manipulating and extracting information from images. There are fairly sophisticated algorithms like CNNs which would be difficult to cover in an introduction, but even these techniques build on some simple tools, such as segmentation and feature detection. I'll cover some of the basic tools in hopes that you'll be able to tackle more advanced techniques with the right vocabulary.


# Topics

- Segmentation
  - Thresholding
  - Clustering
  - Edges...
- Feature detection
  - Edge detection
  - Corner detection
  - Ridge detection
